In [ ]:
import numpy as np


X = np.random.rand(1000,10)
x = []
p = []
c1 = 1
c2 = 1
#Функция
def Function(x):
    return 0.5 * ((x[0])**2 + (x[0] - x[1])**2 + (x[1] - x[2])**2 + (x[2])**2) - x[0]

#Аналитическое значение градиента
def analytic_grad(f, x):
    return np.array([2 * x[0] - x[1] - 1, -x[0] + 2 * x[1] - x[2], - x[1] + 2 * x[2]])

#Численное значение градиента
def num_grad(f, x, h = 10e-4):
    return (f(x - h * np.ones(x.shape)) + f(x + h * np.ones(x.shape))) / 2 / h

#Гессиан для BFGS
def update_hessian(old_hessian, sk, yk):
    ro = 1 / (np.dot(yk.T, sk))
    I = np.eye(old_hessian.shape[0])

    term1 = I - ro * np.outer(sk, yk)
    term2 = I - ro * np.outer(yk, sk)
    term3 = ro * np.outer(sk, sk)

    new_hessian = np.dot(np.dot(term1, old_hessian), term2) + term3

    return new_hessian


# На вход подается словарь S, где s_i = x_k+1 - x_k
# Словарь Y, где y_k = grad_k+1 - grad_k
# Гессиан для L_BFGS
def Hessian(S, Y, grad, H):
    P = np.empty(len(Y))
    alpha = np.empty(len(Y))
    q = grad
    for i in range(len(Y)):
        P[i] = 1/np.dot(Y[i].T, S[i])
        alpha[i] = P[i] * np.dot(S[i].T, q)
        q = q - alpha[i]*Y[i]
    r = np.dot(H, q)
    for i in reversed(range(len(Y))):
        Betta = P[i] * np.dot(Y[i].T, r)
        r = r + S[i] * (alpha[i] - Betta)
    return -r # Возвращает -Hk * grad Fk

def zoom(f, grad, x, p, c1, c2, alpha_low, alpha_high):
    grad_f = grad(f, x)
    while True:
        alpha_j = 0.5 * (alpha_low + alpha_high)
        f_j = f(x + alpha_j * p)
        if f_j > f(x) + c1 * alpha_j * np.dot(grad_f, p) or f_j >= f(x + alpha_low * p):
            alpha_high = alpha_j
        else:
            b = np.dot(grad_f, p)
            grad_f_j = grad(f, x + alpha_j * p)
            a = np.abs(np.dot(grad_f_j, p))
            if np.abs(np.dot(grad_f_j, p)) <= c2 * np.abs(np.dot(grad_f, p)):
                return alpha_j
            if np.dot(grad_f_j, p) * (alpha_high - alpha_low) >= 0:
                alpha_high = alpha_low
            alpha_low = alpha_j

#Поиск параметра альфа
def line_search(f, grad, x, p, c1 = 10e-4, c2 = 0.1, alpha_max = 100, maxiter = 1000):
    alpha_prev = 0
    alpha_cur = (alpha_prev + alpha_max) / 2
    grad_f = grad(f, x)
    i = 1
    while i <= maxiter:
        f_i = f(x + alpha_cur * p)
        if (f_i > f(x) + c1 * alpha_cur * np.dot(grad_f, p)) or (f_i >= f(x + alpha_prev * p) and i > 1):
            return zoom(f, grad, x, p, c1, c2, alpha_prev, alpha_cur)
        grad_alpha_f = grad(f, x + alpha_cur * p)
        if np.abs(np.dot(grad_alpha_f, p)) <= c2 * np.abs(np.dot(grad_f, p)):
            return alpha_cur
        if grad_alpha_f >= 0:
            return zoom(f, grad, x, p, c1, c2, alpha_prev, alpha_cur)
        alpha_prev = alpha_cur
        alpha_cur = (alpha_prev + alpha_max) / 2
        i += 1


def BFGS(f, x_0, maxiter = 1000, eps = 10e-4, type = "analytical"):
    count = 0
    H_k = np.eye(len(x_0))
    x_k = x_0
    if type == "analytical":
        fgrad = analytic_grad
    elif type == "numerical":
        fgrad = num_grad

    fgrad_value = fgrad(f, x_k)

    while np.linalg.norm(fgrad_value) > eps and count < maxiter:

        p_k = -np.dot(H_k, fgrad_value)
        alpha_k = line_search(f, fgrad, x_k, p_k)
        x_next = x_k + alpha_k * p_k
        fgrad_value_next = fgrad(f, x_next)
        H_k = update_hessian(H_k, x_next - x_k, fgrad_value_next - fgrad_value)
        x_k = x_next
        fgrad_value = fgrad_value_next


        count += 1

    return x_k


def L_BFGS(f, x_0, maxiter = 1000, eps = 10e-4, type = "analytical", m = 3):
    count = 0
    H_k = np.eye(len(x_0))
    x_k = x_0
    S = []
    Y = []
    if type == "analytical":
        fgrad = analytic_grad
    elif type == "numerical":
        fgrad = num_grad

    fgrad_value = fgrad(f, x_k)


    while np.linalg.norm(fgrad_value) > eps and count < maxiter:

        p_k = Hessian(S, Y, fgrad_value, H_k)
        alpha_k = line_search(f, fgrad, x_k, p_k)
        x_next = x_k + alpha_k * p_k
        fgrad_value_next = fgrad(f, x_next)
        s = x_next - x_k
        y = fgrad_value_next - fgrad_value
        H_k = np.dot (s, y) / np.dot(y, y) * H_k

        x_k = x_next
        fgrad_value = fgrad_value_next

        if len(S) == m:
            S.pop(0)
            Y.pop(0)
        S.append(s)
        Y.append(y)

        count += 1
    return x_k

x_final = BFGS(f = Function, x_0 = [1, 2, 3])
x_final1 = L_BFGS(f = Function, x_0 = [1, 2, 3])

print (x_final)
print (x_final1)

[0.75013529 0.50007857 0.24997273]
